# NYC Taxi Rides

In this notebook we show some more advanced features that are useful for building composed views, a.k.a., dashboards.

In [1]:
%load_ext autoreload
%autoreload 2

### Download the data

In [2]:
import pandas as pd
from utils import download_file

nyc_data = pd.read_csv('data/nyc-taxi.csv')

for data in nyc_data.iterrows():
    download_file(*data[1])

File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`


### Create tilesets

In [3]:
from higlass.tilesets import cooler

dropoff = {
    'count': cooler('data/nyctaxi-dropoff.count.1m.mcool', name='Total Dropoffs'),
    'passenger_count': cooler('data/nyctaxi-dropoff.passenger_count.mean.1m.mcool', name='Dropoffs Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-dropoff.tip_amount.mean.1m.mcool', name='Dropoffs Mean Tip'),
    'total_amount': cooler('data/nyctaxi-dropoff.total_amount.mean.1m.mcool', name='Dropoffs Mean Total Fair'),
    'trip_distance': cooler('data/nyctaxi-dropoff.trip_distance.mean.1m.mcool', name='Dropoffs Mean Trip Distance'),
}
pickup = {
    'count': cooler('data/nyctaxi-pickup.count.1m.mcool', name='Total Pickups'),
    'passenger_count': cooler('data/nyctaxi-pickup.passenger_count.mean.1m.mcool', name='Pickups Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-pickup.tip_amount.mean.1m.mcool', name='Pickups Mean Tip'),
    'total_amount': cooler('data/nyctaxi-pickup.total_amount.mean.1m.mcool', name='Pickups Mean Total Fair'),
    'trip_distance': cooler('data/nyctaxi-pickup.trip_distance.mean.1m.mcool', name='Pickups Mean Trip Distance'),
}

### Visualize pickups and drop-offs

In [4]:
from higlass import display
from higlass.client import Track, View

nyc_x=[11789206, 11827005]
nyc_y=[15044734, 15081957]

In [5]:
view_config = {
    'width': 6,
    'initialXDomain': nyc_x,
    'initialYDomain': nyc_y,
}

pickups_view = View(
    x=0,
    tracks=[Track('heatmap', tileset=pickup['count'], height=400)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track('heatmap', tileset=dropoff['count'], height=400)],
    **view_config
)

widget, server, viewconf = display([pickups_view, dropoffs_view])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'HbKcMwy4SX6Iq…

### Synchronize view location and zoom level

In [6]:
widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'HbKcMwy4SX6Iq…

### Add crosshairs and tooltips

In [7]:
## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
    }
}
##############################

pickups_view = View(
    x=0,
    tracks=[Track(tileset=pickup['count'], **track_config)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track(tileset=dropoff['count'], **track_config)],
    **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'Ou4uC-RQRfq2-…

### Overview+details

In [8]:
from copy import deepcopy

## Important change ##########
view_config_details = deepcopy(view_config)
view_config_details['y'] = 6
view_config_details['initialXDomain'] = [11820321, 11826194]
view_config_details['initialYDomain'] = [15073235, 15079019]

pickups_view_details = View(
    x=0,
    tracks=[Track(tileset=pickup['count'], **track_config)],
    **view_config_details
)

dropoffs_view_details = View(
    x=6,
    tracks=[Track(tileset=dropoff['count'], **track_config)],
    **view_config_details
)

pickups_view_overview = View(
    x=0,
    tracks=[
        Track(tileset=pickup['count'], **track_config),
        Track('viewport-projection-center', fromViewUid=pickups_view_details.uid),
    ],
    **view_config
)

dropoffs_view_overview = View(
    x=6,
    tracks=[
        Track(tileset=dropoff['count'], **track_config),
        Track('viewport-projection-center', fromViewUid=dropoffs_view_details.uid),
    ],
    **view_config
)
##############################

widget, server, viewconf = display(
    views=[
        pickups_view_overview, dropoffs_view_overview,
        pickups_view_details, dropoffs_view_details
    ],
    location_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
    zoom_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'fB1--ZNuQ-24R…

### Compare different attributes through data operations

In [9]:
"""Dummy data until the `mean.mcool`s are fixed"""

track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True
    }
}

pickups_view = View(
    x=0,
    tracks=[Track(tileset=pickup['count'], **track_config)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track(tileset=dropoff['count'], **track_config)],
    **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'AdWrnc5TTvWo2…

Let's divide pickups by dropoff

In [10]:
from higlass.client import ViewConf
from higlass.viewer import HiGlassDisplay

# This gives us the serverless viewconfig
pickups_dict = pickups_view.to_dict()

# Lets add the server's API endpoint from the previous call
pickups_dict['tracks']['center'][0]['server'] = server.api_address

# With this magic setup we tell HiGlass to divide pickups by dropoffs
pickups_dict['tracks']['center'][0]['data'] = {
    'type': 'divided',
    'children': [{
        'server': server.api_address,
        'tilesetUid': pickup['count'].uuid
    }, {
        'server': server.api_address,
        'tilesetUid': dropoff['count'].uuid
    }]
}

# Let's use a diverging color map
pickups_dict['tracks']['center'][0]['options']['colorRange'] = [
    'rgba(98, 206, 246, 0.85)',
    'rgba(0, 149, 229, 0.5)',
    'rgba(255, 255, 255, 0.01)',
    'rgba(217, 71, 149, 0.5)',
    'rgba(255, 102, 184, 0.85)',
]

pickups_dict['tracks']['center'][0]['options']['name'] = 'Pickups divided by dropoffs'

# Finally we need to convert the dict back to a View object
new_pickups_view = View.from_dict(pickups_dict)

# Finally, we need to create a new view config and pass that into the widget
HiGlassDisplay(viewconf=ViewConf([new_pickups_view]).to_dict())

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'AdWrnc5TTvWo2NcxqX-NPQ', 'tracks': {'top': [], '…

☝️ areas with more pickups are shown in pink and areas with more dropoffs in blue

In [11]:
"""Remove if we can't fix the `.mean.mcool`s"""

## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
    }
}
##############################

pickup_amount_view = View(
    x=0,
    y=0,
    tracks=[Track(tileset=pickup['total_amount'], **track_config)],
    **view_config
)

pickup_dist_view = View(
    x=0,
    y=6,
    tracks=[Track(tileset=pickup['trip_distance'], **track_config)],
    **view_config
)

dropoff_amount_view = View(
    x=6,
    y=0,
    tracks=[Track(tileset=dropoff['total_amount'], **track_config)],
    **view_config
)

dropoff_dist_view = View(
    x=6,
    y=6,
    tracks=[Track(tileset=dropoff['trip_distance'], **track_config)],
    **view_config
)

widget, server, viewconf = display(
    views=[
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    ],
    location_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
    zoom_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'M-cVIhJASz2er…

In [13]:
from higlass.client import ViewConf
from higlass.viewer import HiGlassDisplay

colormap = [
    'rgba(98, 206, 246, 0.85)',
    'rgba(0, 149, 229, 0.5)',
    'rgba(255, 255, 255, 0.01)',
    'rgba(217, 71, 149, 0.5)',
    'rgba(255, 102, 184, 0.85)',
]

# This gives us the serverless viewconfig
pickup_amount_dict = pickup_amount_view.to_dict()
pickup_amount_dict['tracks']['center'][0]['options']['name'] = 'Pickups: Amount by Distance'

# Lets add the server's API endpoint from the previous call
pickup_amount_dict['tracks']['center'][0]['server'] = server.api_address

# With this magic setup we tell HiGlass to divide pickups by dropoffs
pickup_amount_dict['tracks']['center'][0]['data'] = {
    'type': 'divided',
    'children': [{
        'server': server.api_address,
        'tilesetUid': pickup['total_amount'].uuid
    }, {
        'server': server.api_address,
        'tilesetUid': pickup['trip_distance'].uuid
    }]
}

# Let's use the diverging color map
pickup_amount_dict['tracks']['center'][0]['options']['colorRange'] = colormap

# Now we have to recreate the view
pickup_amount_by_dist_view = View.from_dict(pickup_amount_dict)

# And the same thing again for the dropoffs
dropoff_amount_dict = dropoff_amount_view.to_dict()
pickup_amount_dict['tracks']['center'][0]['options']['name'] = 'Dropoffs: Amount by Distance'
dropoff_amount_dict['tracks']['center'][0]['server'] = server.api_address
dropoff_amount_dict['tracks']['center'][0]['data'] = {
    'type': 'divided',
    'children': [{
        'server': server.api_address,
        'tilesetUid': dropoff['total_amount'].uuid
    }, {
        'server': server.api_address,
        'tilesetUid': dropoff['trip_distance'].uuid
    }]
}
dropoff_amount_dict['tracks']['center'][0]['options']['colorRange'] = colormap
dropoff_amount_by_dist_view = View.from_dict(dropoff_amount_dict)

# Finally, we need to create a new view config and pass that into the widget
HiGlassDisplay(viewconf=ViewConf([
        pickup_amount_by_dist_view,
        dropoff_amount_by_dist_view
    ],
    location_syncs=[(
        pickup_amount_by_dist_view,
        dropoff_amount_by_dist_view
    )],
    zoom_syncs=[(
        pickup_amount_by_dist_view,
        dropoff_amount_by_dist_view
    )]
).to_dict())

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'M-cVIhJASz2erV6fWZbkTg', 'tracks': {'top': [], '…

### Superimpose tracks

In [14]:
view_config_together = deepcopy(view_config)
view_config_together['width'] = 12

## Important change ##########
heatmap_track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'backgroundColor': 'transparent',
        'colorRange': [
            'rgba(255, 255, 255, 0.05)',
            'rgba(245, 166,  35, 0.4)',
            'rgba(208,   2,  27, 0.85)',
            'black',
        ],
    }
}
mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqZXB2aWd4NDBmZTIzM3BjdGZudTFob2oifQ.Jnmp1xWJyS4_lRhzrZAFBQ',
        'style': 'light-v9',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
    }
}
##############################

pickups_only = View(
    tracks=[
        Track(**mapbox_track_config),
        Track(tileset=dropoff['count'], **heatmap_track_config),
    ],
    **view_config_together
)

widget, server, viewconf = display([pickups_only])

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'T4FGtJa7TQeAD…

### Finally... let's turn the lights off

In [15]:
heatmap_track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'backgroundColor': 'transparent',
        ## Important change ##########
        'colorRange': [
            'rgba(0, 0, 0, 0.01)',
            'rgba(34, 46, 84, 0.25)',
            'rgba(68, 141, 178, 0.5)',
            'rgba(104, 191, 48, 0.7)',
            'rgba(253, 255, 84, 0.8)',
            'rgba(253, 255, 255, 0.9)',
        ],
        ##############################
        'colorbarBackgroundColor': 'black',
        'colorbarBackgroundOpacity': 0.5,
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    }
}
mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqZXB2aWd4NDBmZTIzM3BjdGZudTFob2oifQ.Jnmp1xWJyS4_lRhzrZAFBQ',
        'style': 'dark-v8',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    }
}

pickups_in_dark = View(
    tracks=[
        Track(**mapbox_track_config),
        Track(tileset=dropoff['count'], **heatmap_track_config),
    ],
    **view_config_together
)

## Important change ##########
widget, server, viewconf = display([pickups_in_dark], dark_mode=True)
##############################

widget

HiGlassDisplay(hg_options={'isDarkTheme': True}, viewconf={'editable': True, 'views': [{'uid': 'ejyuAf-fQgWhcO…